# FAO Analysis - Load, clean, merge data

This notebook is part of a general analysis on [Food and Agriculture Organization (FAO)](https://www.fao.org/home/en/) of the United Nations :  
1. 🚀 Exploration
2. 📁 **Load, clean and merge**
3. 📊 Analysis visuals

🎯 The main goal of this notebook is to test the logic behind loading, cleaning and merging the data, before moving the code into a specific python script that can be reused for further analysis

## Imports, setup

In [1]:
import pandas as pd 
from pathlib import Path
import os

In [2]:
package_dir = Path(os.path.dirname(os.getcwd()))

In [3]:
files = ["Production_Crops_Livestock", "Land_Use", "Emissions_Totals", "Food_Balance"]

## Load data

**``load_one_csv()``**

In [4]:
def load_one_csv(filename, processed=False):
    if processed:
        path = Path(package_dir / f"data/processed/{filename}_processed.csv")
    else:
        path = Path(package_dir / f"data/raw/{filename}.csv")
        
    if not path.exists():
        raise FileNotFoundError(f"File not found: {path}")
        
    return pd.read_csv(path)

**``load_raw()``**

In [5]:
def load_raw():
    """
    Load the required csv files and stores them in pandas DataFrames, returns a python dict with
    csv names as keys and dataframes as values
    
    Args: 
        None
    
    Returns: 
        dict_df (dictionnary of pd.Dataframes): dataframes froms csv raw data
    """
    
    dict_df = {}
    
    for filename in files: 
        try: 
            dict_df[filename] = load_one_csv(filename, processed=False)
            print(f"✅ {filename} loaded.")
        except FileNotFoundError as e: 
            print(f"Error, file not found ({filename}) → {e}")
            raise
        
    return dict_df

In [ ]:
dict_df = load_raw()

To be done in the Python script: replace paths with env variables or define it in config file and load it in the script

## Clean data

**Prod dataframe**

In [ ]:
df_prod = dict_df["Production_Crops_Livestock"]

In [ ]:
df_prod = df_prod[["Area", "Item", "Element", "Year", "Unit", "Value", "Flag"]]
df_prod

In [ ]:
df_prod.isna().sum()

In [ ]:
missing_prod = df_prod[df_prod["Value"].isna()]
missing_prod 

In [ ]:
missing_prod["Flag"].value_counts()

___
From FAO Data documentation : 
- **M**: Missing value (data cannot exist; not applicable)
- **A**: Official figure
- **E**: Estimated value
- **I**: Imputed value
___

Strategy is to keep ``NaN`` values in processed data, but to let the user choose the strategy when loading and processing the data : 
- *keep*: keeping all the ``NaN`` values
- *zero*: setting all ``NaN`` values to ``0``
- *drop*: dropping all the ``NaN`` values

In [ ]:
df_prod.duplicated().sum()

**``clean_data()``**

* Keep only useful columns 
* Manage missing rows: no need
* Drop duplicates if needed

In [6]:
columns_to_keep = {
    "Production_Crops_Livestock": ["Area", "Item", "Element", "Year", "Unit", "Value"],
    "Land_Use": ["Area", "Item", "Element", "Year", "Unit", "Value"],
    "Emissions_Totals": ["Area", "Item", "Element", "Year", "Unit", "Value"],
    "Food_Balance": ["Area", "Item", "Element", "Year", "Unit", "Value"]
}

In [7]:
def clean_data(dict_df):
    # Keep useful columns
    for key, df in dict_df.items():
        dict_df[key] = df[columns_to_keep[key]]
        
    # Missing rows: not taken in this part
    
    # Drop duplicates: not needed
    
    print("✅ Data processed")

    return dict_df

In [ ]:
clean_df = clean_data(dict_df)

**``save_processed()``**

* Check if ``processed``file exists and create one if not 
* Save a csv file for each dataframe

In [8]:
def save_processed(clean_df):
    processed_path = package_dir / "data/processed"
    processed_path.mkdir(parents=True, exist_ok=True) 
    
    for key, df in clean_df.items():
        df.to_csv(processed_path / f"{key}_processed.csv", index=False, encoding="utf-8")
        print(f"✅ {key} processed saved.")

In [ ]:
save_processed(clean_df)

**``load_processed()``**

- If data processed is empty -> ``load_raw()``, ``clean()``, ``save()``
- If data processed exists -> ``pd.read_csv("data/processed/...")``

In [9]:
def load_processed(strategy="keep"):
    
    # Process csv files if not done already
    processed_path = Path(package_dir / "data/processed")
    missing_files = [filename for filename in files if not Path(processed_path / f"{filename}_processed.csv").exists()]
    if missing_files:
        print("🟠 Missing files → processing raw data.")
        dict_df = load_raw()
        clean_df = clean_data(dict_df)
        save_processed(clean_df)
    else: 
        try:
            clean_df = {filename: load_one_csv(filename, processed=True) for filename in files}
            print("✅ Loaded processed data from disk")
        except FileNotFoundError as e: 
            print(f"Error, file not found - {e}")
            raise
    
    # Applying strategy
    strategies = {
        "keep": lambda df: df,
        "zero": lambda df: df.fillna(0),
        "drop": lambda df: df.dropna()
    }
    if strategy not in strategies:
        raise ValueError(f"Unkown strategy {strategy}, please chose from: {', '.join(strategies)}")
    
    clean_df = {key: strategies[strategy](df) for key, df in clean_df.items()}
    
    return clean_df 

In [10]:
clean_df = load_processed()

🟠 Missing files → processing raw data.


/tmp/ipykernel_57547/1169561811.py:10: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(path)


✅ Production_Crops_Livestock loaded.
✅ Land_Use loaded.
✅ Emissions_Totals loaded.
✅ Food_Balance loaded.
✅ Data processed
✅ Production_Crops_Livestock processed saved.
✅ Land_Use processed saved.
✅ Emissions_Totals processed saved.
✅ Food_Balance processed saved.
